## Introduction

Irvine is the largest city in area in Orange County with 65.61 sq miles, and 3rd largest in population with 212,375 people according to census in 2010. 

A number of corporations, particularly in the technology and semiconductor sectors, have their national or international headquarters in Irvine. A lot of them also have office in Bay Area (this analysis will focus on Santa Clara county). During the last couple of years, I had quite a few friends who relocated to Santa Clara county. One big task in the relocation is to find housing, and neighborhood is an important consideration.

Thus, I want to start this analysis on the 15 cities in Santa Clara county and find out which one is the most similar to Irvine.

## Data

Data used in this analysis mainly inclue:the population and demographic related data from wikipedia pages, together with the venue information from foursquare API. 